In [1]:
from __future__ import division  # no need to worry about interger division

from pymongo import MongoClient
import psycopg2 as pg2

from pandas.io.sql import read_sql
import pandas as pd
import numpy as np

from itertools import chain, izip

import matplotlib.pylab as plt
import seaborn as sns

%matplotlib inline

/Users/l/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Users/l/anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [2]:
sns.set(font='Avenir', style='white')

In [4]:
conn = pg2.connect(dbname='satoshi', user='l', host='/tmp')

q = '''
    SELECT * FROM basic_stats2
    WHERE author = 'satoshi';
    '''

df = read_sql(q, conn)

conn.close()

In [5]:
df.shape

(558, 50)

In [6]:
df.groupby('type').describe()

apostrophes     hyphens  paragraphs  periods_nospace  \
type                                                                
email count    18.000000   18.000000          18        18.000000   
      mean      5.000000    7.444444           0         5.555556   
      std       4.214820    8.590160           0         3.617315   
      min       0.000000    0.000000           0         1.000000   
      25%       2.250000    2.000000           0         3.250000   
      50%       4.000000    3.500000           0         5.000000   
      75%       6.000000    8.750000           0         7.000000   
      max      14.000000   29.000000           0        14.000000   
forum count   539.000000  539.000000         539       539.000000   
      mean      2.448980    1.560297           0         2.818182   
      std       2.410227    3.190014           0         2.522089   
      min       0.000000    0.000000           0         0.000000   
      25%       1.000000    0.000000           0         1.000000   
      50%       2.000000    0.000000           0         2.000000   
      75%       4.000000    2.000000           0         4.000000   
      max      20.000000   39.000000           0        17.000000   
paper count     1.000000    1.000000           1         1.000000   
      mean     21.000000   73.000000          23        47.000000   
      std            NaN         NaN         NaN              NaN   
      min      21.000000   73.000000          23        47.000000   
      25%      21.000000   73.000000          23        47.000000   
      50%      21.000000   73.000000          23        47.000000   
      75%      21.000000   73.000000          23        47.000000   
      max      21.000000   73.000000          23        47.000000   

             periods_onespace  periods_twospaces  pronouns_plural  \
type                                                                
email count         18.000000          18.000000        18.000000   
      mean           0.555556           8.611111         0.555556   
      std            0.615699           5.370276         0.783823   
      min            0.000000           1.000000         0.000000   
      25%            0.000000           6.000000         0.000000   
      50%            0.500000           7.500000         0.000000   
      75%            1.000000          10.750000         1.000000   
      max            2.000000          21.000000         2.000000   
forum count        539.000000         539.000000       539.000000   
      mean           2.810761           0.003711         0.565863   
      std            2.800922           0.060858         1.219735   
      min            0.000000           0.000000         0.000000   
      25%            1.000000           0.000000         0.000000   
      50%            2.000000           0.000000         0.000000   
      75%            4.000000           0.000000         1.000000   
      max           22.000000           1.000000        12.000000   
paper count          1.000000           1.000000         1.000000   
      mean          85.000000           0.000000        24.000000   
      std                 NaN                NaN              NaN   
      min           85.000000           0.000000        24.000000   
      25%           85.000000           0.000000        24.000000   
      50%           85.000000           0.000000        24.000000   
      75%           85.000000           0.000000        24.000000   
      max           85.000000           0.000000        24.000000   

             pronouns_singular  sentences_lower  sentences_punkt     ...      \
type                                                                 ...       
email count          18.000000        18.000000        18.000000     ...       
      mean            1.833333         0.055556        15.555556     ...       
      std             2.455486         0.235702         8.311737     ...       
      min             0

In [7]:
df_sum = df.groupby('type').sum()

# Sentences:

### Observations:
* hard to extract paragraph from e-mail and forum posts
    * sentence/paragraph not comparable between paper/email vs. forum (mostly answering people's question)
* word count: 
    * paper ~ 3000; 
    * email ~ 5000; 
    * forum ~ 50000
* space after period: 
    * paper: ~one space
    * email: ~two spaces
    * forum: ~one space
    * periods_twospaces/periods_onespace -- scale goes from 0 to inf -- not a good feature
* sentences:
    * upper + lower ~= 88% of punkt -- use punkt

    
### Preliminary set of features author attribution (satoshi vs. satoshi):
* periods_onespace/sentences_punkt
* periods_twospaces/sentences_punkt
* words/sentence_punkt

In [11]:
df_sum[['words',
        'periods_nospace',
        'periods_onespace',
        'periods_twospaces']]

words  periods_nospace  periods_onespace  periods_twospaces
type                                                              
email   5036              100                10                155
forum  49875             1519              1515                  2
paper   3007               47                85                  0

In [12]:
df_sum[['sentences_punkt',
        'sentences_upper',
        'sentences_lower']]

sentences_punkt  sentences_upper  sentences_lower
type                                                    
email              280              256                1
forum             3127             2641               74
paper              132              116                1

In [13]:
(df_sum['sentences_upper'] + df_sum['sentences_lower']) / df_sum['sentences_punkt']

type
email    0.917857
forum    0.868244
paper    0.886364
dtype: float64

In [14]:
df_sum['words/sentences_punkt']       = df_sum['words'] / df_sum['sentences_punkt']
df_sum['words/sentences_upper_lower'] = df_sum['words'] / (df_sum['sentences_upper'] + df_sum['sentences_lower'])

In [15]:
df_sum[['words/sentences_punkt',
        'words/sentences_upper_lower']]

words/sentences_punkt  words/sentences_upper_lower
type                                                     
email              17.985714                    19.595331
forum              15.949792                    18.370166
paper              22.780303                    25.700855

# Words:

### Observations:
* all types of writing: words_US >= words_GB -- how much of the difference is because of the dictionaries used?

### Preliminary set of features for author attribution (satoshi vs. satoshi):
* words_nostop/words
* words_primaryvb/words
* words_to/words
* words_verb/words
* words_noun/words
* words_desc/words
* words_det/words
* words_conj/words
* words_gb/words_us

In [16]:
df_sum['words_nostop/words']    = df_sum['words_nostop']    / df_sum['words']
df_sum['words_primaryvb/words'] = df_sum['words_primaryvb'] / df_sum['words']
df_sum['words_to/words']        = df_sum['words_to']        / df_sum['words']

In [17]:
df_sum[['words_nostop/words',
        'words_primaryvb/words', 
        'words_to/words']]

words_nostop/words  words_primaryvb/words  words_to/words
type                                                            
email            0.522041               0.053415        0.033360
forum            0.515469               0.060150        0.035268
paper            0.538078               0.049218        0.039574

In [18]:
word_types = {'verb': ['vb', 'vbd', 'vbn', 'vbp', 'vbz'],
              'noun': ['nn', 'nns', 'nnp', 'nnps', 'prp', 'pp$'],                          # noun, pronoun
              'desc': ['jj', 'jjr', 'jjs', 'rb', 'rbr', 'rbs', 'wdt', 'wp', 'wp$', 'wrb'], # adj, adv, "which what how, etc"
              'det' : ['dt', 'pdt'],                                                       # determiner
              'conj': ['cc', 'in']}                                                        # conjugate-like

cnames  = set(df.columns)

col_map = dict()
for key, lst in word_types.iteritems():
    cname = 'words_%s' % key
    col_map[cname] = set('words_' + c for c in lst)
    df_sum[cname]  = df_sum[list(cnames & col_map[cname])].sum(axis=1)
    df_sum['%s/words' % cname] = df_sum[cname] / df_sum['words']

In [19]:
df_sum[[a+b for a,b in izip(col_map.keys(), ['/words']*len(col_map))]]


words_det/words  words_noun/words  words_verb/words  words_conj/words  \
type                                                                           
email         0.125496          0.268864          0.160643          0.143963   
forum         0.101734          0.257243          0.172912          0.132030   
paper         0.137346          0.273695          0.168274          0.152311   

       words_desc/words  
type                     
email          0.161239  
forum          0.157133  
paper          0.141004

In [20]:
df_sum[['words_us', 'words_gb']]

words_us  words_gb
type                     
email      4889      4871
forum     46718     45843
paper      2962      2962

In [21]:
df_sum['words_us/words']    = df_sum['words_us'] / df_sum['words']
df_sum['words_gb/words']    = df_sum['words_gb'] / df_sum['words']
df_sum['words_gb/words_us'] = df_sum['words_gb'] / df_sum['words_us']

In [22]:
df_sum[['words_us/words', 'words_gb/words', 'words_gb/words_us']]

words_us/words  words_gb/words  words_gb/words_us
type                                                    
email        0.970810        0.967236           0.996318
forum        0.936702        0.919158           0.981271
paper        0.985035        0.985035           1.000000

# Spellings:


In [25]:
client = MongoClient()
db  = client['satoshi']
tbl = db['word-lists2']

In [26]:
dic = dict()

for wt in ['paper', 'email', 'forum']:
    
    query_results = tbl.find( { 'author' : 'satoshi',
                                'type'   : wt         } )

    # cursor only in one direction!
    words = [(result['misspellings'], result['us_spellings'], result['gb_spellings']) for result in query_results]
    dic['%s_misspell' % wt], dic['%s_US' % wt], dic['%s_GB' % wt] = zip(*words)


In [27]:
# total words
for wt in ['paper', 'email', 'forum']:
    print wt+':', len(list(chain(*dic['%s_US' % wt]))), len(list(chain(*dic['%s_GB' % wt])))

paper: 2 3
email: 21 17
forum: 525 90


In [28]:
# unique words
for wt in ['paper', 'email', 'forum']:
    print wt+':', len(set(chain(*dic['%s_US' % wt]))), len(set(chain(*dic['%s_GB' % wt])))

paper: 2 3
email: 10 15
forum: 51 38


# Word list... should have lemmatized!!
* vbs in different tense...
* singular/plural nouns
* lemmatize **DOES NOT** mean change word to another word with same meaning e.g. Honda -> car
* add lemmatizer and start all over again

In [119]:
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = []
for word in 'if you red thinking of'.split():
    # filter -- if word can be many POS, select the POS matching the filter
    v = lemmatizer.lemmatize(word, pos='v')  
    if v == word:
        v = lemmatizer.lemmatize(word, pos='n')  
    words += v.encode('ascii'),
words

['if', 'you', 'red', 'think', 'of']